In [ ]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
import pymongo

In [ ]:
# # Initialize PyMongo to work with MongoDBs

# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)

In [ ]:
#  # Define database and collection
    
# db = client.news_db
# collection = db.articles

In [ ]:
 # Setup splinter
    
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url = 'https://redplanetscience.com/'

browser.visit(url)

In [ ]:
html = browser.html
soup = bs(html, 'html.parser')

# soup.prettify()

In [ ]:
title = soup.find("div", class_="content_title").text

title

In [ ]:
paragraph = soup.find("div", class_="article_teaser_body").text

paragraph

In [ ]:
url = 'https://spaceimages-mars.com/'

browser.visit(url)

In [ ]:
html = browser.html
soup = bs(html, 'html.parser')

# soup.prettify()

In [ ]:
browser.links.find_by_partial_text('FULL IMAGE').click()

In [ ]:
html = browser.html
soup = bs(html, 'html.parser')
image_rel_url = soup.find("img", class_="fancybox-image")["src"]

In [ ]:
browser.url

In [ ]:
featured_image_url = f"{browser.url}{image_rel_url}"
featured_image_url

In [ ]:
url = "https://galaxyfacts-mars.com/"

In [ ]:
tables = pd.read_html(url)

In [ ]:
tables[1]

In [ ]:
df = tables[1]

In [ ]:
df = df.rename(columns={0:"Characteristic", 1:"Data"})

In [ ]:
df

In [ ]:
df["Characteristic"] =  df["Characteristic"].str.rstrip(":")

In [ ]:
df

In [ ]:
html_table = df.to_html()

In [ ]:
html_table.replace('\n', '') 

In [ ]:
df.to_html('table.html') 

In [ ]:
url = 'https://marshemispheres.com/'

browser.visit(url)
hemi_landing_url = browser.url

In [ ]:
html = browser.html
soup = bs(html, 'html.parser')

# soup.prettify()

In [ ]:
image_tags = soup.find_all("img", class_="thumb")

In [ ]:
image_tags

In [ ]:
tif_title = image_tags[0]["alt"].replace(" Enhanced thumbnail", "")
tif_title

In [ ]:
links_found = browser.find_by_css('h3').links.find_by_partial_text('Hemisphere Enhanced')
links_found

In [ ]:
for link in links_found:
    img_link = link["href"]
    print(img_link)
#     browser.visit(img_link)
#     html = browser.html
#     soup = bs(html, 'html.parser')
#     tif_url = soup.find("img", class_="wide-image")
#     print(tif_url["src"])
    
    

In [ ]:
browser.visit(links_found[0]["href"])

In [ ]:
html = browser.html
soup = bs(html, 'html.parser')

soup.prettify()

In [ ]:
# finds .jpg not .tif

wide_img_tag = soup.find("img", class_="wide-image")

In [ ]:
wide_img_rel_url = wide_img_tag["src"]

In [ ]:
wide_img_url = f"{hemi_landing_url}{wide_img_rel_url}"

In [ ]:
wide_img_url

In [ ]:
# now find .tif

downloads_tags = soup.find("div", class_="downloads")

In [ ]:
downloads_tags

In [ ]:
img_tags = downloads_tags.find_all("a")

In [ ]:
img_tags

In [ ]:
tif_url = f'{hemi_landing_url}{img_tags[1]["href"]}'

In [ ]:
tif_url

In [ ]:
{"title":tif_title, "img_url":tif_url}

In [ ]:
browser.quit()